# Setup
- Install alphafold3, colabfold
- Copy model weights from Google Drive (adjust path & give permission when asked)

In [ ]:
%%shell
pip install --quiet git+https://github.com/jurgjn/alphafold3-polymer-bonds.git
ALPHAFOLD3_VERSION="2e2ffc1⁠"
source <(curl -fsSL https://raw.githubusercontent.com/jurgjn/alphafold3-polymer-bonds/refs/heads/main/colab/install)

In [ ]:
# Copy weights from Google Drive to the default location at /root/models/af3.bin.zst
from google.colab import drive
drive.mount('/mnt/gdrive')
!mkdir -p /root/models/
!cp /mnt/gdrive/MyDrive/alphafold3/models/af3.bin.zst /root/models/af3.bin.zst
!md5sum /root/models/af3.bin.zst
drive.flush_and_unmount()

In [ ]:
# Standard imports
import argparse, collections, copy, gzip, json, os, os.path, re, string, sys
import numpy as np
# Helper functions for manipulating AlphaFold 3 input jsons: https://github.com/google-deepmind/alphafold3/blob/main/docs/input.md
from alphafold3_polymer_bonds.alphafold3_io import colab_data_pipeline, init_input_json, read_input_json, print_input_json, write_input_json

# 1DF6 cycloviolacin O1

## Sequence-only baseline

In [ ]:
# Fetch peptide sequence from the PDB
!curl -fsSL https://www.rcsb.org/fasta/entry/1DF6

In [ ]:
js = init_input_json('SCVYIPCTVTALLGCSCSNRVCYNGIPCAE')
js['name'] = '1DF6_baseline'
js = colab_data_pipeline(js)
print_input_json(js)
write_input_json(js, '1DF6_baseline_input/{}.json')

In [ ]:
%%shell
source /alphafold3_venv/bin/activate
/app/alphafold/run_alphafold3_autogpu.sh \
  --input_dir=/content/1DF6_baseline_input/ \
  --output_dir=/content/1DF6_baseline_output/ \
  --norun_data_pipeline

## Cyclic with GLU30 as ligand

In [ ]:
js = init_input_json('SCVYIPCTVTALLGCSCSNRVCYNGIPCA')
js['name'] = '1DF6_cyclic'
js['sequences'].append(
  collections.OrderedDict([('ligand',
    collections.OrderedDict([
        ('id', 'B'),
        ('ccdCodes', ['GLU']),
    ])
  )])
)
js['bondedAtomPairs'] = [
  [['A', 29, 'C'], ['B', 1, 'N']],
  [['B', 1, 'C'], ['A', 1, 'N']],
]
js = colab_data_pipeline(js)
#print_json(js)
write_input_json(js, '1DF6_cyclic_input/{}.json')

In [ ]:
%%shell
source /alphafold3_venv/bin/activate
/app/alphafold/run_alphafold3_autogpu.sh \
  --input_dir=/content/1DF6_cyclic_input \
  --output_dir=/content/1DF6_cyclic_output \
  --norun_data_pipeline

# 6OQ1 branched tri-ubiquitin

## Sequence-only baseline

In [ ]:
!curl -fsSL https://www.rcsb.org/fasta/entry/6OQ1

In [ ]:
js = init_input_json(
  'MQIFVKTLTGRTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGCQLEDGRTLSDYNIQRESTLHLVLRLRGG',
  'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGRQLEDGRTLSDYNIQKESTLHLVLRLRGG',
  'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGD',
)
js['name'] = '6OQ1_baseline'
js = colab_data_pipeline(js)
write_input_json(js, '6OQ1_baseline_input/{}.json')

In [ ]:
%%shell
source /alphafold3_venv/bin/activate
/app/alphafold/run_alphafold3_autogpu.sh \
  --input_dir=/content/6OQ1_baseline_input \
  --output_dir=/content/6OQ1_baseline_output \
  --norun_data_pipeline

## Branching with polybonds script

In [ ]:
js = init_input_json(
  'MQIFVKTLTGRTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGCQLEDGRTLSDYNIQRESTLHLVLRLRGG',
  'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGRQLEDGRTLSDYNIQKESTLHLVLRLRGG',
  'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGD',
)
js['name'] = '6OQ1_polybonds'
js['bondedAtomPairs'] = [
  [['C', 11, 'NZ'], ['A', 76, 'C']],
  [['C', 48, 'NZ'], ['B', 76, 'C']],
]
write_input_json(js, '6OQ1_polybonds_bondedAtoms/{}.json')

In [ ]:
!alphafold3_polymer_bonds --source_path 6OQ1_polybonds_bondedAtoms/6OQ1_polybonds.json --output_path 6OQ1_polybonds_encoded/6OQ1_polybonds.json

In [ ]:
js = read_input_json('6OQ1_polybonds_encoded/6OQ1_polybonds.json')
js = colab_data_pipeline(js)
print_input_json(js)
write_input_json(js, '6OQ1_polybonds_input/6OQ1_polybonds.json')

In [ ]:
%%shell
source /alphafold3_venv/bin/activate
/app/alphafold/run_alphafold3_autogpu.sh \
  --input_dir=/content/6OQ1_polybonds_input \
  --output_dir=/content/6OQ1_polybonds_output \
  --norun_data_pipeline